In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
import numpy as np
X = np.load('/gdrive/My Drive/local_features_inception_avg_pooling.npy')


In [3]:
X1 = np.load('/gdrive/My Drive/global_features_inception_avg_pooling.npy')

In [4]:
X = X.reshape((3200, 1000))
X1 = X1.reshape((3200, 1000))

In [5]:
X_concatenated = np.concatenate((X, X1), axis=1)

In [6]:
b_label = [0] * 800
n_label = [1] * 800
inv_label = [2] * 800
ins_label = [3] * 800
y = b_label + n_label + inv_label + ins_label

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_concatenated, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_concatenated, y, test_size=0.2, random_state=12345, stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#Standard Scale features
scaler = StandardScaler()
scaler.fit(X_train)
X_sc_train = scaler.transform(X_train)
X_sc_test = scaler.transform(X_test)

In [ ]:
# Usign Standard Scaler
pca = PCA(n_components=500)
X_train_transformed = pca.fit_transform(X_train)
X_test_transformed = pca.transform(X_test)

In [ ]:
import lightgbm as lgb

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
# fit model no training data
Xgb_clf = XGBClassifier()
Xgb_clf.fit(X_train_transformed, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
XGB_y_preds = Xgb_clf.predict(X_test_transformed)

In [ ]:
X_train_transformed.shape

(2560, 500)

In [ ]:
clf = lgb.LGBMClassifier()
# clf.fit(X_train_transformed, y_train)

clf.fit(X_train, y_train)

LGBMClassifier()

In [ ]:
LGB_y_preds = clf.predict(X_test)
# LGB_y_preds = clf.predict(X_sc_test)

In [ ]:
from sklearn.metrics import classification_report
# print(classification_report(y_test, SVM_y_preds, labels=["Benign", "Normal", "Invasive", "Insitu"]))
# print(classification_report(y_test, SVM_y_preds))
# print(classification_report(y_test, KNN_y_preds, labels=["Benign", "Normal", "Invasive", "Insitu"]))
print(classification_report(y_test, LGB_y_preds))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78       160
           1       0.82      0.78      0.79       160
           2       0.86      0.83      0.84       160
           3       0.81      0.81      0.81       160

    accuracy                           0.81       640
   macro avg       0.81      0.81      0.81       640
weighted avg       0.81      0.81      0.81       640



#Check voting

In [7]:
# Checking XGB

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

# reshape array
X_reshaped = X_concatenated.reshape((400, 8, 2000))
b_label = [0] * 100
n_label = [1] * 100
inv_label = [2] * 100
ins_label = [3] * 100
y_new = b_label + n_label + inv_label + ins_label
y_new = np.array(y_new)
# perform k-fold cross-validation
report_list = []
kf = KFold(n_splits=5, shuffle=True, random_state=12345 )
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345 )

j = 0 


for train_index, test_index in skf.split(X_reshaped,y_new):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    X_train_reshaped = X_train.reshape((320*8,2000))
    X_test_reshaped = X_test.reshape((80*8,2000))
    scaler = StandardScaler()
    scaler.fit(X_train_reshaped)
    X_sc_train = scaler.transform(X_train_reshaped)
    X_sc_test = scaler.transform(X_test_reshaped)
    # Usign Standard Scaler
    pca = PCA(n_components=500)
    X_train_transformed = pca.fit_transform(X_sc_train)
    X_test_transformed = pca.transform(X_sc_test)
    clf = XGBClassifier()
    # Repeat each element 8 times and expand the dimensions
    # print(y_train)
    y_train = np.repeat(y_train, 8)
    y_train = y_train.reshape((320*8))
    # print(y_train)
    clf.fit(X_train_transformed, y_train)
    LGB_y_preds = clf.predict(X_test_transformed)
    y_pred = []
    for i in range(len(y_test)):
      start = 8*i
      end = 8*i +8
      # get the labels of the current group
      group_pred = LGB_y_preds[start:end]
      # use voting to predict the label of the group
      pred_label = np.argmax(np.bincount(group_pred))
      y_pred.append(pred_label)
    
    y_test_new = np.repeat(y_test, 8)
    y_test_new = y_test_new.reshape((80*8))
    report = classification_report(y_test, y_pred)
    report_list.append(report)
    print(report)
    j+=1
    print(j)



    

              precision    recall  f1-score   support

           0       0.34      0.50      0.41        20
           1       0.37      0.35      0.36        20
           2       0.44      0.40      0.42        20
           3       0.50      0.35      0.41        20

    accuracy                           0.40        80
   macro avg       0.41      0.40      0.40        80
weighted avg       0.41      0.40      0.40        80

1
              precision    recall  f1-score   support

           0       0.40      0.40      0.40        20
           1       0.54      0.65      0.59        20
           2       0.43      0.45      0.44        20
           3       0.47      0.35      0.40        20

    accuracy                           0.46        80
   macro avg       0.46      0.46      0.46        80
weighted avg       0.46      0.46      0.46        80

2
              precision    recall  f1-score   support

           0       0.60      0.60      0.60        20
           1     

In [9]:
results = [
    {'precision': [0.34, 0.37, 0.44, 0.50],
     'recall': [0.50, 0.35, 0.40, 0.35],
     'f1-score': [0.41, 0.36, 0.42, 0.41]},
    {'precision': [0.40, 0.54, 0.43, 0.47],
     'recall': [0.40, 0.65, 0.45, 0.35],
     'f1-score': [0.40, 0.59, 0.44, 0.40]},
    {'precision': [0.60, 0.72, 0.67, 0.72],
     'recall': [0.60, 0.65, 0.80, 0.65],
     'f1-score': [0.60, 0.68, 0.73, 0.68]},
    {'precision': [0.48, 0.63, 0.50, 0.64],
     'recall': [0.60, 0.60, 0.55, 0.45],
     'f1-score': [0.53, 0.62, 0.52, 0.53]},
    {'precision': [0.48, 0.65, 0.52, 0.67],
     'recall': [0.60, 0.55, 0.60, 0.50],
     'f1-score': [0.53, 0.59, 0.56, 0.57]}
]

# Compute the average precision, recall, and f1-score
avg_precision = sum(results[i]['precision'][j] for i in range(len(results)) for j in range(4)) / (len(results) * 4)
avg_recall = sum(results[i]['recall'][j] for i in range(len(results)) for j in range(4)) / (len(results) * 4)
avg_f1_score = sum(results[i]['f1-score'][j] for i in range(len(results)) for j in range(4)) / (len(results) * 4)

# Print the results
print(f"Average Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")
print(f"Average F1-Score: {avg_f1_score:.2f}")


Average Precision: 0.54
Average Recall: 0.53
Average F1-Score: 0.53


In [10]:
accuracies = [0.41, 0.46, 0.68, 0.56, 0.58]

# Calculate average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print("average Accuracy: " + str(average_accuracy))

average Accuracy: 0.538


# Two class classification

In [16]:


# Convert to two-class classification
y_pred_new = np.where(np.isin(y_pred, [0, 1]), 0, 1)
y_test_new = np.where(np.isin(y_test, [0, 1]), 0, 1)

print(classification_report(y_test_new, y_pred_new))

              precision    recall  f1-score   support

           0       0.65      0.80      0.72        40
           1       0.74      0.57      0.65        40

    accuracy                           0.69        80
   macro avg       0.70      0.69      0.68        80
weighted avg       0.70      0.69      0.68        80



#KNN

In [13]:
# Checking XGB

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

# reshape array
X_reshaped = X_concatenated.reshape((400, 8, 2000))
b_label = [0] * 100
n_label = [1] * 100
inv_label = [2] * 100
ins_label = [3] * 100
y_new = b_label + n_label + inv_label + ins_label
y_new = np.array(y_new)
# perform k-fold cross-validation
report_list = []
kf = KFold(n_splits=5, shuffle=True, random_state=12345 )
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345 )

j = 0 


for train_index, test_index in skf.split(X_reshaped,y_new):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    X_train_reshaped = X_train.reshape((320*8,2000))
    X_test_reshaped = X_test.reshape((80*8,2000))
    scaler = StandardScaler()
    scaler.fit(X_train_reshaped)
    X_sc_train = scaler.transform(X_train_reshaped)
    X_sc_test = scaler.transform(X_test_reshaped)
    # Usign Standard Scaler
    pca = PCA(n_components=500)
    X_train_transformed = pca.fit_transform(X_sc_train)
    X_test_transformed = pca.transform(X_sc_test)
    clf = KNeighborsClassifier()
    # Repeat each element 8 times and expand the dimensions
    # print(y_train)
    y_train = np.repeat(y_train, 8)
    y_train = y_train.reshape((320*8))
    # print(y_train)
    clf.fit(X_train_transformed, y_train)
    LGB_y_preds = clf.predict(X_test_transformed)
    y_pred = []
    for i in range(len(y_test)):
      start = 8*i
      end = 8*i +8
      # get the labels of the current group
      group_pred = LGB_y_preds[start:end]
      # use voting to predict the label of the group
      pred_label = np.argmax(np.bincount(group_pred))
      y_pred.append(pred_label)
    
    y_test_new = np.repeat(y_test, 8)
    y_test_new = y_test_new.reshape((80*8))
    report = classification_report(y_test, y_pred)
    report_list.append(report)
    print(report)
    j+=1
    print(j)



    

              precision    recall  f1-score   support

           0       0.37      0.70      0.48        20
           1       0.43      0.30      0.35        20
           2       0.50      0.45      0.47        20
           3       0.50      0.25      0.33        20

    accuracy                           0.42        80
   macro avg       0.45      0.42      0.41        80
weighted avg       0.45      0.42      0.41        80

1
              precision    recall  f1-score   support

           0       0.32      0.65      0.43        20
           1       0.77      0.50      0.61        20
           2       0.37      0.35      0.36        20
           3       0.43      0.15      0.22        20

    accuracy                           0.41        80
   macro avg       0.47      0.41      0.40        80
weighted avg       0.47      0.41      0.40        80

2
              precision    recall  f1-score   support

           0       0.42      0.65      0.51        20
           1     

In [ ]:
results = [
    {'precision': [0.59, 0.68, 0.79, 0.59],
     'recall': [0.50, 0.75, 0.75, 0.65],
     'f1-score': [0.54, 0.71, 0.77, 0.62]},
    {'precision': [0.50, 0.65, 0.62, 0.54],
     'recall': [0.50, 0.75, 0.40, 0.65],
     'f1-score': [0.50, 0.70, 0.48, 0.59]},
    {'precision': [0.73, 0.76, 1.00, 0.77],
     'recall': [0.80, 0.80, 0.75, 0.85],
     'f1-score': [0.76, 0.78, 0.86, 0.81]},
    {'precision': [0.50, 0.65, 0.88, 0.60],
     'recall': [0.60, 0.65, 0.70, 0.60],
     'f1-score': [0.55, 0.65, 0.78, 0.60]},
    {'precision': [0.50, 0.48, 0.76, 0.70],
     'recall': [0.45, 0.60, 0.65, 0.70],
     'f1-score': [0.47, 0.53, 0.70, 0.70]}
]

# Compute the average precision, recall, and f1-score
avg_precision = sum(results[i]['precision'][0] for i in range(len(results))) / len(results)
avg_recall = sum(results[i]['recall'][0] for i in range(len(results))) / len(results)
avg_f1_score = sum(results[i]['f1-score'][0] for i in range(len(results))) / len(results)

# Print the results
print(f"Average Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")
print(f"Average F1-Score: {avg_f1_score:.2f}")


Average Precision: 0.56
Average Recall: 0.57
Average F1-Score: 0.56


In [14]:
accuracies = [0.45, 0.47, 0.61, 0.50, 0.49]

# Calculate average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print("average Accuracy: " + str(average_accuracy))

average Accuracy: 0.5039999999999999


In [17]:


# Convert to two-class classification
y_pred_new = np.where(np.isin(y_pred, [0, 1]), 0, 1)
y_test_new = np.where(np.isin(y_test, [0, 1]), 0, 1)

print(classification_report(y_test_new, y_pred_new))

              precision    recall  f1-score   support

           0       0.65      0.80      0.72        40
           1       0.74      0.57      0.65        40

    accuracy                           0.69        80
   macro avg       0.70      0.69      0.68        80
weighted avg       0.70      0.69      0.68        80



# SVM

In [18]:
# Checking XGB

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# reshape array
X_reshaped = X_concatenated.reshape((400, 8, 2000))
b_label = [0] * 100
n_label = [1] * 100
inv_label = [2] * 100
ins_label = [3] * 100
y_new = b_label + n_label + inv_label + ins_label
y_new = np.array(y_new)
# perform k-fold cross-validation
report_list = []
kf = KFold(n_splits=5, shuffle=True, random_state=12345 )
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345 )

j = 0 


for train_index, test_index in skf.split(X_reshaped,y_new):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    X_train_reshaped = X_train.reshape((320*8,2000))
    X_test_reshaped = X_test.reshape((80*8,2000))
    scaler = StandardScaler()
    scaler.fit(X_train_reshaped)
    X_sc_train = scaler.transform(X_train_reshaped)
    X_sc_test = scaler.transform(X_test_reshaped)
    # Usign Standard Scaler
    pca = PCA(n_components=500)
    X_train_transformed = pca.fit_transform(X_sc_train)
    X_test_transformed = pca.transform(X_sc_test)
    clf = svm.SVC()
    # Repeat each element 8 times and expand the dimensions
    # print(y_train)
    y_train = np.repeat(y_train, 8)
    y_train = y_train.reshape((320*8))
    # print(y_train)
    clf.fit(X_train_transformed, y_train)
    LGB_y_preds = clf.predict(X_test_transformed)
    y_pred = []
    for i in range(len(y_test)):
      start = 8*i
      end = 8*i +8
      # get the labels of the current group
      group_pred = LGB_y_preds[start:end]
      # use voting to predict the label of the group
      pred_label = np.argmax(np.bincount(group_pred))
      y_pred.append(pred_label)
    
    y_test_new = np.repeat(y_test, 8)
    y_test_new = y_test_new.reshape((80*8))
    report = classification_report(y_test, y_pred)
    report_list.append(report)
    print(report)
    j+=1
    print(j)



    

              precision    recall  f1-score   support

           0       0.45      0.75      0.57        20
           1       0.56      0.45      0.50        20
           2       0.50      0.45      0.47        20
           3       0.54      0.35      0.42        20

    accuracy                           0.50        80
   macro avg       0.51      0.50      0.49        80
weighted avg       0.51      0.50      0.49        80

1
              precision    recall  f1-score   support

           0       0.48      0.50      0.49        20
           1       0.48      0.60      0.53        20
           2       0.41      0.35      0.38        20
           3       0.59      0.50      0.54        20

    accuracy                           0.49        80
   macro avg       0.49      0.49      0.49        80
weighted avg       0.49      0.49      0.49        80

2
              precision    recall  f1-score   support

           0       0.64      0.70      0.67        20
           1     

In [22]:
results = [
    {'precision': [0.37, 0.43, 0.50, 0.50],
     'recall': [0.70, 0.30, 0.45, 0.25],
     'f1-score': [0.48, 0.35, 0.47, 0.33]},
    {'precision': [0.32, 0.77, 0.37, 0.43],
     'recall': [0.65, 0.50, 0.35, 0.15],
     'f1-score': [0.43, 0.61, 0.36, 0.22]},
    {'precision': [0.42, 0.67, 0.52, 0.85],
     'recall': [0.65, 0.50, 0.55, 0.55],
     'f1-score': [0.51, 0.57, 0.54, 0.67]},
    {'precision': [0.39, 0.35, 0.53, 0.71],
     'recall': [0.75, 0.35, 0.40, 0.25],
     'f1-score': [0.52, 0.35, 0.46, 0.37]},
    {'precision': [0.36, 0.60, 0.44, 0.54],
     'recall': [0.70, 0.30, 0.40, 0.35],
     'f1-score': [0.47, 0.40, 0.42, 0.42]}
]

# Compute the average precision, recall, and f1-score
avg_precision = sum(results[i]['precision'][0] for i in range(len(results))) / len(results)
avg_recall = sum(results[i]['recall'][0] for i in range(len(results))) / len(results)
avg_f1_score = sum(results[i]['f1-score'][0] for i in range(len(results))) / len(results)

# Print the results
print(f"Average Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")
print(f"Average F1-Score: {avg_f1_score:.2f}")


Average Precision: 0.37
Average Recall: 0.69
Average F1-Score: 0.48


In [20]:
accuracies = [0.51, 0.49, 0.66, 0.57, 0.66]

# Calculate average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print("average Accuracy: " + str(average_accuracy))

average Accuracy: 0.5780000000000001


In [21]:


# Convert to two-class classification
y_pred_new = np.where(np.isin(y_pred, [0, 1]), 0, 1)
y_test_new = np.where(np.isin(y_test, [0, 1]), 0, 1)

print(classification_report(y_test_new, y_pred_new))

              precision    recall  f1-score   support

           0       0.74      0.88      0.80        40
           1       0.85      0.70      0.77        40

    accuracy                           0.79        80
   macro avg       0.80      0.79      0.79        80
weighted avg       0.80      0.79      0.79        80



# LGB results

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# reshape array
X_reshaped = X_concatenated.reshape((400, 8, 2000))
b_label = [0] * 100
n_label = [1] * 100
inv_label = [2] * 100
ins_label = [3] * 100
y_new = b_label + n_label + inv_label + ins_label
y_new = np.array(y_new)
# perform k-fold cross-validation
report_list = []
kf = KFold(n_splits=5, shuffle=True, random_state=12345 )
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345 )

j = 0 


for train_index, test_index in skf.split(X_reshaped,y_new):
    X_train, X_test = X_reshaped[train_index], X_reshaped[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    X_train_reshaped = X_train.reshape((320*8,2000))
    X_test_reshaped = X_test.reshape((80*8,2000))
    scaler = StandardScaler()
    scaler.fit(X_train_reshaped)
    X_sc_train = scaler.transform(X_train_reshaped)
    X_sc_test = scaler.transform(X_test_reshaped)
    # Usign Standard Scaler
    pca = PCA(n_components=500)
    X_train_transformed = pca.fit_transform(X_sc_train)
    X_test_transformed = pca.transform(X_sc_test)
    clf = lgb.LGBMClassifier()
    # Repeat each element 8 times and expand the dimensions
    # print(y_train)
    y_train = np.repeat(y_train, 8)
    y_train = y_train.reshape((320*8))
    # print(y_train)
    clf.fit(X_train_reshaped, y_train)
    LGB_y_preds = clf.predict(X_test_reshaped)
    y_pred = []
    for i in range(len(y_test)):
      start = 8*i
      end = 8*i +8
      # get the labels of the current group
      group_pred = LGB_y_preds[start:end]
      # use voting to predict the label of the group
      pred_label = np.argmax(np.bincount(group_pred))
      y_pred.append(pred_label)
    
    y_test_new = np.repeat(y_test, 8)
    y_test_new = y_test_new.reshape((80*8))
    report = classification_report(y_test, y_pred)
    report_list.append(report)
    print(report)
    j+=1
    print(j)



    

              precision    recall  f1-score   support

           0       0.46      0.60      0.52        20
           1       0.47      0.45      0.46        20
           2       0.50      0.50      0.50        20
           3       0.53      0.40      0.46        20

    accuracy                           0.49        80
   macro avg       0.49      0.49      0.49        80
weighted avg       0.49      0.49      0.49        80

1
              precision    recall  f1-score   support

           0       0.41      0.35      0.38        20
           1       0.50      0.65      0.57        20
           2       0.41      0.45      0.43        20
           3       0.60      0.45      0.51        20

    accuracy                           0.48        80
   macro avg       0.48      0.47      0.47        80
weighted avg       0.48      0.47      0.47        80

2
              precision    recall  f1-score   support

           0       0.68      0.65      0.67        20
           1     